[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danmillr/places-platforms/blob/main/02_Index/03_Suitability.ipynb)

In [ ]:
# MaxEnt (Presence-Only Prediction) Analysis Template

# BLOCK 1: Introduction
"""
This notebook demonstrates how to run a MaxEnt (Maximum Entropy) presence-only prediction analysis using occurrence points and explanatory variables.
The goal is to model the likelihood of presence across a geographic area, even when absence data is unavailable. While we will not use the original MaxEnt Java software directly, this notebook walks through a Python-based approximation using logistic regression.
This template is helpful for urban analysis, ecological modeling, or environmental planning projects where students only have point observations of a phenomenon.
"""

# BLOCK 2: Load Occurrence Points
"""
In this block, we load a dataset of observed presence points.
This could be locations of tree species, street vendors, or public benches — anything where presence-only data is available.
"""
import geopandas as gpd

# Replace with your own file
occurrence_path = "data/occurrence_points.geojson"
occurrences = gpd.read_file(occurrence_path)
occurrences = occurrences.to_crs("EPSG:4326")  # Ensure WGS84 for alignment
occurrences.head()

# BLOCK 3: Load and Prepare Explanatory Rasters
"""
Load the raster layers that will be used to explain/predict presence.
These rasters represent environmental or spatial variables (e.g., NDVI, elevation, proximity to roads).
Make sure all rasters are clipped to the same extent and projected correctly.
"""
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

# Example: Load raster datasets
raster_paths = {
    "elevation": "data/elevation.tif",
    "ndvi": "data/ndvi.tif",
    "dist_to_roads": "data/dist_to_roads.tif"
}

rasters = {}
for key, path in raster_paths.items():
    src = rasterio.open(path)
    rasters[key] = src.read(1, masked=True)
    print(f"Loaded {key} with shape {rasters[key].shape}")
    show(rasters[key], title=key)

# BLOCK 4: Sample Raster Values at Occurrence Points
"""
We extract values from the rasters at the point locations to get the variable values
at each occurrence (presence) point.
"""
from rasterio.sample import sample

coords = [(x,y) for x, y in zip(occurrences.geometry.x, occurrences.geometry.y)]

sampled_values = {}
for key, path in raster_paths.items():
    with rasterio.open(path) as src:
        values = list(src.sample(coords))
        sampled_values[key] = [v[0] for v in values]  # Assume single band

import pandas as pd
X = pd.DataFrame(sampled_values)
y = np.ones(len(X))  # Presence = 1
X.head()

# BLOCK 5: Sample Background Points for Absence
"""
MaxEnt uses presence-only data but needs background samples for comparison.
This block creates artificial "pseudo-absence" or "background" points sampled from the same region.
"""
from shapely.geometry import box
from sklearn.utils import resample

bbox = occurrences.total_bounds
extent = box(*bbox)
background_points = gpd.GeoSeries([extent.centroid.buffer(0.1*i).centroid for i in range(1, len(X)+1)])
background_coords = [(pt.x, pt.y) for pt in background_points]

background_samples = {}
for key, path in raster_paths.items():
    with rasterio.open(path) as src:
        values = list(src.sample(background_coords))
        background_samples[key] = [v[0] for v in values]

X_bg = pd.DataFrame(background_samples)
y_bg = np.zeros(len(X_bg))  # Background = 0

# Combine datasets
X_full = pd.concat([X, X_bg], ignore_index=True)
y_full = np.concatenate([y, y_bg])

# BLOCK 6: Train MaxEnt-Like Model in Python
"""
We use logistic regression as a proxy for MaxEnt to estimate the probability of presence.
This is a simplification and suitable for teaching concepts when the MaxEnt GUI is not available.
"""
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:, 1]
print("AUC Score:", roc_auc_score(y_test, y_pred))

# BLOCK 7: Predict Presence Probability Raster
"""
Now we apply the trained model across the entire raster stack to generate a prediction raster
indicating the probability of presence at each pixel.
"""
def stack_rasters(raster_paths):
    arrays = []
    for path in raster_paths.values():
        with rasterio.open(path) as src:
            arrays.append(src.read(1, masked=True))
    return np.stack(arrays, axis=-1)

stacked = stack_rasters(raster_paths)

flat_stack = stacked.reshape(-1, stacked.shape[-1])
predicted = model.predict_proba(flat_stack)[:, 1]
predicted_raster = predicted.reshape(stacked.shape[:2])

# BLOCK 8: Save Prediction to GeoTIFF
"""
Save the predicted probability raster to a GeoTIFF file for further visualization in GIS software.
"""
meta = src.meta.copy()
meta.update({"count": 1, "dtype": "float32"})

with rasterio.open("output/predicted_presence.tif", "w", **meta) as dst:
    dst.write(predicted_raster.astype("float32"), 1)

# BLOCK 9: Visualize Prediction
"""
Display the predicted presence probability map.
Lighter colors (yellow/green) indicate higher probability of presence.
"""
plt.imshow(predicted_raster, cmap="viridis")
plt.colorbar(label="Presence Probability")
plt.title("Predicted Presence Map")
plt.show()

# BLOCK 10: Wrap-up & Extensions
"""
- You can export your point and raster inputs to the MaxEnt Java application for additional features (e.g., jackknife plots).

"""
